<a href="https://colab.research.google.com/github/offdocudev/Machine-Learning/blob/master/Gen_adaptive_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import math
from IPython.display import clear_output

# DATA - https://www.kaggle.com/c/digit-recognizer/data
# Kaggle 
! pip install kaggle
! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
# Or manually copy credentials in kaggle.json file 

!  echo '{"username":"offdocudev","key":"a6893bc3aa38aec8a688959083e82644"}' > /root/.kaggle/kaggle.json


! cat /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json

#################### DOWNLOAD AND UNZIP FILE SAVED IN DRIVE ####################

!pip install -U -q PyDrive

# HERE YOUR FILE ID ( GET IT WITH THE SHARING URL: https://drive.google.com/open?id=1Soh3zXLXt2lT7b_3FcWWyeOCC7SnOxK0 )
zip_id = '1Soh3zXLXt2lT7b_3FcWWyeOCC7SnOxK0'

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)




Using TensorFlow backend.


{"username":"offdocudev","key":"a6893bc3aa38aec8a688959083e82644"}
    100% |████████████████████████████████| 993kB 20.1MB/s 


In [0]:
from keras.layers import Input,Dense,Flatten,Dropout,Reshape,Concatenate,BatchNormalization,Conv2D,Conv2DTranspose,Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
import keras.backend as K
from keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.optimizers import Adam

In [0]:
import sys

In [0]:
def get_generator(input_layer,target_layer):
  merged_layer = Concatenate()([input_layer,target_layer])
  hid = Dense(128 * 8 * 8,activation='relu')(merged_layer)
  hid = BatchNormalization(momentum = 0.9)(hid)
  hid = LeakyReLU(alpha = 0.1)(hid)
  hid = Reshape((8,8,128))(hid)
  
  
  hid = Conv2D(128,kernel_size = 5,strides = 1,padding = 'same' )(hid)
  hid = BatchNormalization(momentum = 0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
  
  hid = Conv2DTranspose(128,kernel_size = 4,strides =2,padding= 'same')(hid)
  hid = BatchNormalization(momentum = 0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
  
  hid = Conv2D(128,kernel_size=5,strides = 1,padding ='same')(hid)
  hid = BatchNormalization(momentum = 0.9)(hid)
  hid = LeakyReLU(alpha =0.1)(hid)
  
  hid = Conv2DTranspose(128,kernel_size=4,strides=2,padding = 'same')(hid)
  hid = BatchNormalization(momentum =0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
  
  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
                      
  hid = Conv2D(3, kernel_size=5, strides=1, padding="same")(hid)
  out = Activation("tanh")(hid)

  model = Model(inputs=[input_layer, target_layer], outputs=out)
  model.summary()
  
  return model, out

In [0]:
def get_discriminator(input_layer,target_layer):
  hid = Conv2D(128,kernel_size= 3,strides =1,padding = 'same')(input_layer)
  hid = BatchNormalization(momentum = 0.9)(hid)
  hid = LeakyReLU(alpha =0.1)(hid)
  
  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Flatten()(hid)
  
  merged_layer = Concatenate()([hid,target_layer])
  hid = Dense(512,activation='relu')(merged_layer)
  
  out = Dense(1,activation='sigmoid')(hid)
  model = Model(inputs = [input_layer,target_layer],outputs = out)
  model.summary()
  
  return model,out
  

In [0]:
from keras.preprocessing import image

def one_Hot(y):
  z = np.zeros((len(y),10))
  idx = np.arange(len(y))
  z[idx,y] = 1
  return z
  

In [0]:
def generate_noise(n_samples,noise_dim):
  X = np.random.normal(0,1,size = (n_samples,noise_dim))
  return X

def generate_random_labels(n):
  y = np.random.choice(10,n)
  y = one_Hot(y)
  return y


In [0]:
tags = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [0]:
from keras.preprocessing import image

def one_hot_encode(y):
  z = np.zeros((len(y), 10))
  idx = np.arange(len(y))
  z[idx, y] = 1
  return z

def generate_noise(n_samples, noise_dim):
  X = np.random.normal(0, 1, size=(n_samples, noise_dim))
  return X

def generate_random_labels(n):
  y = np.random.choice(10, n)
  y = one_hot_encode(y)
  return y

In [0]:
def show_samples(batchidx):
  fig,axs = plt.subplots(5,6,figsize = (10,10))
  plt.subplots_adjust(hspace=0.3,wspace=0.1)
  
  for classlabel in range(10):
    row = int(classlabel /2)
    col = (classlabel % 2 ) *3
    lbls = one_hot_encode([classlabel ] * 3)
    noise = generate_noise(3,100)
    gen_imgs = generator.predict([noise,lbls])
    
    for i in range(3):
      img = image.array_to_img(gen_imgs[i],scale = True)
      axs[row,col+i].imshow(img)
      axs[row,i+col].axis('off')
      
      if i==1:
        axs[row,i+col].set_title(tags[classlabel])
        
plt.show()
plt.close()
    

In [0]:
img_input = Input(shape=(32,32,3))
disc_condition_input = Input(shape=(10,))

discriminator, disc_out = get_discriminator(img_input, disc_condition_input)
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False

noise_input = Input(shape=(100,))
gen_condition_input = Input(shape=(10,))
generator, gen_out = get_generator(noise_input, gen_condition_input)

gan_input = Input(shape=(100,))
x = generator([gan_input, gen_condition_input])
gan_out = discriminator([x, disc_condition_input])
gan = Model(inputs=[gan_input, gen_condition_input, disc_condition_input], output=gan_out)
gan.summary()

gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 128)  3584        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 128)  512         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 32, 32, 128)  0           batch_normalization_1[0][0]      
_____________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("mo...)`
  app.launch_new_instance()


In [0]:
BATCH_SIZE = 512

# # Get training images
(X_train, y_train), (X_test, _) = cifar10.load_data()

# Normalize data
X_train = (X_train - 127.5) / 127.5

# 1hot encode labels
y_train = one_hot_encode(y_train[:,0])

print ("Training shape: {}".format(X_train.shape))
 
num_batches = int(X_train.shape[0]/BATCH_SIZE)

170500096/170498071 [==============================] - 44s 0us/step
Training shape: (50000, 32, 32, 3)


In [0]:
exp_replay = []

In [0]:
N_EPOCHS = 10
for epoch in range(N_EPOCHS):

  cum_d_loss = 0.
  cum_g_loss = 0.
  
  for batch_idx in range(num_batches):
    # Get the next set of real images to be used in this iteration
    images = X_train[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]
    labels = y_train[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]

    noise_data = generate_noise(BATCH_SIZE, 100)
    random_labels = generate_random_labels(BATCH_SIZE)
    # We use same labels for generated images as in the real training batch
    generated_images = generator.predict([noise_data, labels])

    # Train on soft targets (add noise to targets as well)
    noise_prop = 0.05 # Randomly flip 5% of targets
    
    # Prepare labels for real data
    true_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
    true_labels[flipped_idx] = 1 - true_labels[flipped_idx]
    
    # Train discriminator on real data
    d_loss_true = discriminator.train_on_batch([images, labels], true_labels)

    # Prepare labels for generated data
    gene_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
    gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]
    
    # Train discriminator on generated data
    d_loss_gene = discriminator.train_on_batch([generated_images, labels], gene_labels)
    
    # Store a random point for experience replay
    r_idx = np.random.randint(BATCH_SIZE)
    exp_replay.append([generated_images[r_idx], labels[r_idx], gene_labels[r_idx]])
    
    #If we have enough points, do experience replay
    if len(exp_replay) == BATCH_SIZE:
      generated_images = np.array([p[0] for p in exp_replay])
      labels = np.array([p[1] for p in exp_replay])
      gene_labels = np.array([p[2] for p in exp_replay])
      expprep_loss_gene = discriminator.train_on_batch([generated_images, labels], gene_labels)
      exp_replay = []
      break
    
    d_loss = 0.5 * np.add(d_loss_true, d_loss_gene)
    cum_d_loss += d_loss

    # Train generator
    noise_data = generate_noise(BATCH_SIZE, 100)
    random_labels = generate_random_labels(BATCH_SIZE)
    g_loss = gan.train_on_batch([noise_data, random_labels, random_labels], np.zeros((BATCH_SIZE, 1)))
    cum_g_loss +=  g_loss

  print('\tEpoch: {}, Generator Loss: {}, Discriminator Loss: {}'.format(epoch+1, cum_g_loss/num_batches, cum_d_loss/num_batches))
  show_samples("epoch" + str(epoch))

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


	Epoch: 1, Generator Loss: 2.150850628512268, Discriminator Loss: [0.51420313 0.        ]
